# 

In [1]:
import os
import sys
import json
from hdlConvertor import HdlConvertor
from hdlConvertorAst.language import Language
from hdlConvertorAst.to.vhdl import ToVhdl2008
# from hdlConvertorAst.to.verilog import ToVerilog2005
from hdlConvertorAst.hdlAst import HdlValueInt, HdlValueId

In [2]:
h = HdlConvertor()
h.parse?

Signature:
h.parse(
    filenames,
    language,
    incdirs,
    encoding='utf-8',
    hierarchyOnly=False,
    debug=True,
)
Docstring:
:param filenames: sequence of filenames or filename
:type filename: Union[str, List[str]]
:param language: hdlConvertor.language.Language enum value
:param incdirs: list of include directories
:param encoding: character encoding of input data
:param hierarchyOnly: if True only names of components and modules are parsed
:param debug: if True the debug logging is enabled
:return: HdlContext instance
Type:      method

In [3]:
from hdlConvertor import HdlConvertor
from hdlConvertorAst.to.vhdl.vhdl2008 import ToVhdl2008
from hdlConvertorAst.hdlAst import HdlModuleDec, HdlValueId, HdlValueInt
import io
import json

class GenericExtractor:
    """
    Extracts and manipulates VHDL generics using the hdlConvertorAst library
    """
    def __init__(self, file_path, include_dirs=[], language=Language.VHDL_2008):
        self.file_path = file_path
        self.converter = HdlConvertor()
        self.ast = self.converter.parse([file_path], language, incdirs=include_dirs)
        
    def find_generics(self):
        """
        Extract all generics from parsed VHDL file
        
        Returns:
            Dictionary mapping entity names to their generic parameters
        """
        entities_with_generics = {}
        
        # Traverse AST to find entities
        for obj in self.ast.objs:
            if hasattr(obj, 'dec') and obj.dec is not None:
                print(obj)
                entity_dec = obj.dec
                
                # Check if it's an entity declaration with a name
                if hasattr(entity_dec, 'name'):
                    entity_name = entity_dec.name
                    
                    # Look for generics (stored as params in hdlConvertorAst)
                    if hasattr(entity_dec, 'params') and entity_dec.params:
                        # Use StringIO to capture the VHDL output for each generic
                        entity_generics = []
                        
                        for param in entity_dec.params:
                            # Extract generic information
                            generic_info = {
                                'name': param.name,
                                'type': self._extract_type(param),
                                'default_value': self._extract_default_value(param)
                            }
                            entity_generics.append(generic_info)
                        
                        if entity_generics:
                            entities_with_generics[entity_name] = entity_generics
        
        return entities_with_generics
    
    def _extract_type(self, param):
        """
        Extract type information from a parameter node using ToVhdl2008
        """
        if not hasattr(param, 'type') or param.type is None:
            return "Unknown"
        
        # Use ToVhdl2008 to render the type properly
        output = io.StringIO()
        vhdl_writer = ToVhdl2008(output)
        vhdl_writer.visit_type(param.type)
        
        return output.getvalue()
    
    def _extract_default_value(self, param):
        """
        Extract default value from a parameter node using ToVhdl2008
        """
        if not hasattr(param, 'value') or param.value is None:
            return None
        
        # Use ToVhdl2008 to render the default value properly
        output = io.StringIO()
        vhdl_writer = ToVhdl2008(output)
        vhdl_writer.visit_iHdlExpr(param.value)
        
        return output.getvalue()
    
    def generate_modified_ast(self, modified_generics):
        """
        Create a new AST with modified generic values
        
        Args:
            modified_generics: Dictionary of entity names to 
                              dictionaries of generic names and new values
        """
        # Clone the AST (simplified approach - in practice would need deep copy)
        modified_ast = self.ast
        
        # Traverse and modify generics
        for obj in modified_ast.objs:
            if hasattr(obj, 'dec') and obj.dec is not None:
                entity_dec = obj.dec
                
                if hasattr(entity_dec, 'name') and entity_dec.name in modified_generics:
                    generic_changes = modified_generics[entity_dec.name]
                    
                    if hasattr(entity_dec, 'params'):
                        for param in entity_dec.params:
                            if param.name in generic_changes:
                                new_value = generic_changes[param.name]
                                
                                # Convert to appropriate HdlValue type
                                if isinstance(new_value, int):
                                    param.value = HdlValueInt(new_value, None)
                                else:
                                    param.value = HdlValueId(str(new_value))
        
        return modified_ast
    
    def generate_vhdl(self, ast=None, output_file=None):
        """
        Regenerate VHDL from AST with original comments preserved
        """
        if ast is None:
            ast = self.ast
            
        output = io.StringIO()
        vhdl_writer = ToVhdl2008(output)
        
        for obj in ast.objs:
            vhdl_writer.visit_main_obj(obj)
        
        vhdl_text = output.getvalue()
        
        if output_file:
            with open(output_file, 'w') as f:
                f.write(vhdl_text)
        
        return vhdl_text

def print_vhdl_generics(file_path, output_file=None):
    """
    Extract and print generics from a VHDL file
    """
    extractor = GenericExtractor(file_path)
    generics_dict = extractor.find_generics()
    
    if not generics_dict:
        print("No generics found in the VHDL file.")
        return
    
    # Print summary
    print(f"Found generics in {len(generics_dict)} entities:")
    for entity_name, generics in generics_dict.items():
        print(f"\nEntity: {entity_name}")
        print("Generics:")
        for g in generics:
            default = f" (default: {g['default_value']})" if g['default_value'] is not None else ""
            print(f"  - {g['name']}: {g['type']}{default}")
    
    # Save to JSON
    if output_file:
        with open(output_file, "w") as f:
            json.dump(generics_dict, f, indent=2)
        print(f"\nExported generics to {output_file}")
    
    return generics_dict

def modify_vhdl_generics(file_path, modified_generics, output_file):
    """
    Modify generics in a VHDL file and output to a new file
    
    Args:
        file_path: Path to the original VHDL file
        modified_generics: Dictionary of entities to dictionaries of generic names and new values
        output_file: Path to write the modified VHDL
    """
    extractor = GenericExtractor(file_path)
    modified_ast = extractor.generate_modified_ast(modified_generics)
    vhdl_text = extractor.generate_vhdl(modified_ast, output_file)
    
    print(f"Generated modified VHDL with updated generics: {output_file}")
    return vhdl_text

In [4]:

def expand_objs(ast):
    if hasattr(ast, 'objs'):
        for o in ast.objs:
            expand_objs(o)
    else:
        print(ast)
        print("\n\n NEXT OBJECT \n\n\n")
        
expand_objs(ast)

NameError: name 'ast' is not defined

In [27]:
from hdlConvertorAst.to.hdl_ast_visitor import HdlAstVisitor
from hdlConvertorAst.hdlAst import HdlValueId, HdlOp

class TestTraverse(HdlAstVisitor):
    """
    Make port names lower case HDL AST

    :note: this is just a stupid rewrite of all variable names to lower case
    """
    
    def visit_param(self, o):
        # print(dir(o))
        # print(o.direction)
        # print(type(o))
        ...

        
    def visit_HdlIdDef(self, o):
        print(type(o))
        # print(f"GENERIC??? {o}")
        # print(dir(o))
        # print(o.is_const)
        # if o.is_const:
            # print(o)
        o.name = o.name.lower()

    # def visit_HdlStmProcess(self, o):
    #     if o.sensitivity:
    #         o.sensitivity = [self.visit_iHdlExpr(s) for s in o.sensitivity]
    #     self.visit_iHdlStatement(o.body)

    # def visit_iHdlExpr(self, o):
    #     if isinstance(o, HdlValueId):
    #         return HdlValueId(o.val.lower(), obj=o.obj)
    #     elif isinstance(o, HdlOp):
    #         o.ops = [self.visit_iHdlExpr(s) for s in o.ops]
    #         return o
    #     else:
    #         return super(PortNamesToLowerCase, self).visit_iHdlExpr(o)

    # def visit_HdlStmIf(self, o):
    #     o.cond = self.visit_iHdlExpr(o.cond)
    #     print(o)

    # def visit_HdlStmAssign(self, o):
    #     o.src = self.visit_iHdlExpr(o.src)
    #     o.dst = self.visit_iHdlExpr(o.dst)


In [55]:
from hdlConvertor import HdlConvertor
import graphviz
import os
import uuid

class VhdlHierarchyVisualizer:
    def __init__(self):
        self.graph = graphviz.Digraph(
            'VHDL Hierarchy', 
            comment='VHDL Module Hierarchy with Generics',
            format='png'
        )
        # Improve readability with hierarchical layout
        self.graph.attr(rankdir='TB', size='14,10', ratio='fill', nodesep='0.5')
        self.graph.attr('node', shape='box', style='filled', margin='0.2')
        self.node_counter = 0
        self.seen_nodes = {}

    def parse_file(self, file_path):
        """Parse a VHDL file and return the AST"""
        converter = HdlConvertor()
        ast = converter.parse([file_path], Language.VHDL_2008, [])
        return ast
    
    
    def visualize(self, ast, output_file="vhdl_hierarchy"):
        """Create a visualization of the VHDL hierarchy"""
        # Add root node
        root_id = self._add_node("Context", "VHDL Design Units", "lightgrey")
        
        # Process all top-level design units
        if hasattr(ast, 'objs') and ast.objs:
            for obj in ast.objs:
                self._process_design_unit(obj, root_id)
                
        # Render the graph WITHOUT viewing (removes xdg-open dependency)
        try:
            # Set view=False to avoid using xdg-open
            self.graph.render(output_file, view=False)
            print(f"Graph rendered successfully to {output_file}.png")
            return f"{output_file}.png"
        except Exception as e:
            print(f"Error rendering graph: {e}")
            return None
    
    def _process_design_unit(self, unit, parent_id):
        """Process a VHDL design unit (entity, architecture, etc.)"""
        # Determine unit type and create appropriate node
        if hasattr(unit, 'dec') and unit.dec is not None:
            # Entity declaration
            if hasattr(unit.dec, 'name'):
                unit_name = unit.dec.name
                unit_id = self._add_node("Entity", f"Entity: {unit_name}", "lightblue")
                self.graph.edge(parent_id, unit_id)
                
                # Process generics specially (highlighted)
                if hasattr(unit.dec, 'params') and unit.dec.params:
                    generics_id = self._add_node("Generics", "Generic Parameters", "gold")
                    self.graph.edge(unit_id, generics_id)
                    
                    for generic in unit.dec.params:
                        generic_value = self._extract_value(generic)
                        generic_id = self._add_node("Generic", 
                                                    f"{generic.name}: {self._extract_type(generic)}\n= {generic_value}",
                                                    "lightyellow")
                        self.graph.edge(generics_id, generic_id)
                
                # Process ports
                if hasattr(unit.dec, 'ports') and unit.dec.ports:
                    ports_id = self._add_node("Ports", "Ports", "lightgreen")
                    self.graph.edge(unit_id, ports_id)
                    
                    for port in unit.dec.ports:
                        port_id = self._add_node("Port", 
                                                 f"{port.name}: {port.direction} {self._extract_type(port)}", 
                                                 "palegreen")
                        self.graph.edge(ports_id, port_id)
                        
                        # Check for generic-dependent port width
                        if self._is_generic_dependent(port):
                            self.graph.edge(port_id, ports_id, label="generic-width", style="dashed", color="red")
        
        # Process architecture if present
        if hasattr(unit, 'def_') and unit.def_ is not None:
            arch = unit.def_
            if hasattr(arch, 'name') and hasattr(arch, 'module_name'):
                arch_id = self._add_node("Architecture", 
                                        f"Architecture: {arch.name}\nof {arch.module_name}", 
                                        "lightcoral")
                self.graph.edge(parent_id, arch_id)
                
                # Process internal signals
                if hasattr(arch, 'declarations') and arch.declarations:
                    signals_id = self._add_node("Signals", "Internal Signals", "lightpink")
                    self.graph.edge(arch_id, signals_id)
                    
                    for decl in arch.declarations:
                        if hasattr(decl, 'is_signal') and decl.is_signal:
                            signal_id = self._add_node("Signal", 
                                                      f"{decl.name}: {self._extract_type(decl)}", 
                                                      "pink")
                            self.graph.edge(signals_id, signal_id)
                            
                            # Check for generic-dependent signal
                            if self._is_generic_dependent(decl):
                                self.graph.edge(signal_id, signals_id, 
                                               label="generic-dep", 
                                               style="dashed", 
                                               color="red")
                
                # Process component instances - submodules
                self._process_component_instances(arch, arch_id)
    
    def _process_component_instances(self, arch, parent_id):
        """Process component instances (submodules)"""
        if hasattr(arch, 'body') and arch.body:
            instances_id = self._add_node("Instances", "Component Instances", "lightskyblue")
            self.graph.edge(parent_id, instances_id)
            
            for stmt in arch.body:
                if hasattr(stmt, 'is_component_instance') and stmt.is_component_instance:
                    inst_id = self._add_node("Instance", 
                                           f"{stmt.name}: {stmt.component_name}", 
                                           "skyblue")
                    self.graph.edge(instances_id, inst_id)
                    
                    # Show generic map
                    if hasattr(stmt, 'generic_map') and stmt.generic_map:
                        gmap_id = self._add_node("GenericMap", 
                                               "Generic Map", 
                                               "gold")
                        self.graph.edge(inst_id, gmap_id)
                        
                        for mapping in stmt.generic_map:
                            map_id = self._add_node("Mapping", 
                                                  f"{mapping.formal}: {self._extract_expr(mapping.actual)}", 
                                                  "lightyellow")
                            self.graph.edge(gmap_id, map_id)
    
    def _is_generic_dependent(self, node):
        """Check if a node has a type or value that depends on generics"""
        # This is a simplified check - would need more complex parsing for real code
        if hasattr(node, 'type'):
            type_str = str(node.type)
            return '(' in type_str and ')' in type_str and '-' in type_str
        return False
    
    def _extract_type(self, node):
        """Extract type information from a node"""
        if hasattr(node, 'type') and node.type is not None:
            if hasattr(node.type, 'name'):
                return node.type.name
            return str(node.type)
        return "unknown"
    
    def _extract_value(self, node):
        """Extract default value from a node"""
        if hasattr(node, 'value') and node.value is not None:
            if hasattr(node.value, 'val'):
                return node.value.val
            return str(node.value)
        return "N/A"
    
    def _extract_expr(self, expr):
        """Convert expression to string representation"""
        if expr is None:
            return "N/A"
        if hasattr(expr, 'val'):
            return str(expr.val)
        return str(expr)
    
    def _add_node(self, node_type, label, color):
        """Add a node to the graph with appropriate styling"""
        node_id = f"node_{self.node_counter}"
        self.node_counter += 1
        self.graph.node(node_id, label, fillcolor=color)
        return node_id

def visualize_vhdl_hierarchy(file_path, output_file=None):
    """
    Visualize the hierarchical structure of a VHDL file
    with special emphasis on generics and their dependencies
    """
    if output_file is None:
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = f"{base_name}_hierarchy"
        
    visualizer = VhdlHierarchyVisualizer()
    ast = visualizer.parse_file(file_path)
    return visualizer.visualize(ast, output_file)


In [92]:
#VHDL test
TEST_DIR = os.path.join("..", "tests", "vhdl")

filenames = [os.path.join(TEST_DIR, "common_add_sub.vhd"), "fft_r2_par.vhd"]
include_dirs = []

ast = HdlConvertor().parse(
    filenames=filenames[1],
    language=Language.VHDL_2008,
    incdirs=include_dirs,
    hierarchyOnly=False
)
TestTraverse().visit_HdlContext(ast)
serializer = ToVhdl2008(sys.stdout)
# for obj in ast.objs:
#     serializer.visit_main_obj(obj)
# code = serializer.visit_main_obj(ast)
# code;

<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.HdlIdDef'>
<class 'hdlConvertorAst.hdlAst._defs.Hdl

fft_r2_par.vhd:171:4: VhdlProcessParser.visitAttribute_declaration Conversion to Python object not implemented
    ...attributekeep_hierarchy:string;...
fft_r2_par.vhd:172:4: VhdlProcessParser.visitAttribute_specification Conversion to Python object not implemented
    ...attributekeep_hierarchyofstr:architectureis"yes";...


In [93]:
for obj in ast.objs:
    print(obj)

{'__class__': 'HdlLibrary',
 'name': {'__class__': 'str', 'val': 'ieee'},
 'position': (41, 9, 41, 12)}

{'__class__': 'HdlLibrary',
 'name': {'__class__': 'str', 'val': 'common_pkg_lib'},
 'position': (41, 15, 41, 28)}

{'__class__': 'HdlLibrary',
 'name': {'__class__': 'str', 'val': 'common_components_lib'},
 'position': (41, 31, 41, 51)}

{'__class__': 'HdlLibrary',
 'name': {'__class__': 'str', 'val': 'casper_adder_lib'},
 'position': (41, 54, 41, 69)}

{'__class__': 'HdlLibrary',
 'name': {'__class__': 'str', 'val': 'casper_requantize_lib'},
 'position': (41, 72, 41, 92)}

{'__class__': 'HdlLibrary',
 'name': {'__class__': 'str', 'val': 'r2sdf_fft_lib'},
 'position': (41, 95, 41, 107)}

{'__class__': 'HdlImport',
 'path': {'__class__': 'list',
          'items': ['IEEE', 'std_logic_1164', {'__class__': 'HdlAll'}]},
 'position': (42, 5, 42, 27)}

{'__class__': 'HdlImport',
 'path': {'__class__': 'list',
          'items': ['common_pkg_lib', 'common_pkg', {'__class__': 'HdlAll'}]},


In [72]:
graphviz.Digraph?

Init signature:
graphviz.Digraph(
    name: Optional[str] = None,
    comment: Optional[str] = None,
    filename=None,
    directory=None,
    format: Optional[str] = None,
    engine: Optional[str] = None,
    encoding: Optional[str] = 'utf-8',
    graph_attr=None,
    node_attr=None,
    edge_attr=None,
    body=None,
    strict: bool = False,
    *,
    renderer: Optional[str] = None,
    formatter: Optional[str] = None,
) -> None
Docstring:     
Directed graph source code in the DOT language.

Args:
    name: Graph name used in the source code.
    comment: Comment added to the first line of the source.
    filename: Filename for saving the source
        (defaults to ``name`` + ``'.gv'``).
    directory: (Sub)directory for source saving and rendering.
    format: Rendering output format (``'pdf'``, ``'png'``, ...).
    engine: Layout command used (``'dot'``, ``'neato'``, ...).
    renderer: Output renderer used (``'cairo'``, ``'gd'``, ...).
    formatter: Output formatter used (`

In [88]:
import os
import uuid
from hdlConvertor import HdlConvertor
import graphviz

class VhdlAstVisualizer:
    """
    Visualize VHDL Abstract Syntax Trees from hdlConvertor with recursive traversal
    """
    def __init__(self):
        self.graph = graphviz.Digraph(
            'VHDL AST', 
            comment='VHDL Module Hierarchy',
            format='svg'
        )
        self.graph.attr(rankdir='TB', size='16,12', ratio='fill')
        self.node_counter = 0
        self.seen_nodes = {}  # Avoid cycles in visualization
        
    def parse_file(self, file_path):
        """Parse a VHDL file and return the AST"""
        converter = HdlConvertor()
        ast = converter.parse([file_path], Language.VHDL_2008, [])
        return ast
    
    def visualize(self, ast, output_file="vhdl_ast"):
        """Create a visualization of the VHDL AST"""
        self.graph.attr('node', 
                      shape='box', 
                      style='filled,rounded', 
                      margin='0.15',
                      fontsize='11',
                      fontname='Arial')
                      
        # Configure edge defaults
        self.graph.attr('edge',
                      fontsize='9',
                      fontname='Arial',
                      arrowsize='0.6')
        # Add root node
        root_id = self._add_node("HdlContext", "VHDL Design Units", "lightgrey")
        
        # Process all top-level objects
        if hasattr(ast, 'objs') and ast.objs:
            for obj in ast.objs:
                self._process_node(obj, root_id)
                
        # Render the graph WITHOUT viewing (fixes xdg-open dependency issue)
        try:
            self.graph.render(output_file, view=False)
            print(f"Graph rendered successfully to {output_file}.svg")
            return f"{output_file}.svg"
        except Exception as e:
            print(f"Error rendering graph: {e}")
            return None
    
    def _process_node(self, node, parent_id):
        """
        Recursively process an AST node and its children
        
        Args:
            node: The current AST node
            parent_id: ID of the parent node in the graph
        """
        # Skip if already processed (avoid cycles)
        node_hash = id(node)
        if node_hash in self.seen_nodes:
            self.graph.edge(parent_id, self.seen_nodes[node_hash], style="dashed")
            return self.seen_nodes[node_hash]
            
        # Get node type and create a label
        node_type = node.__class__.__name__ if hasattr(node, '__class__') else type(node).__name__
        if hasattr(node, '__class__') and hasattr(node.__class__, '__name__'):
            node_type = node.__class__.__name__
        else:
            node_type = str(type(node)).split("'")[1]
            
        # Create a label with node attributes
        label = self._create_node_label(node)
        
        # Determine node color based on type
        color = self._get_node_color(node_type)
        
        # Add node to graph
        node_id = self._add_node(node_type, label, color)
        self.seen_nodes[node_hash] = node_id
        
        # Connect to parent
        self.graph.edge(parent_id, node_id)
        
        # Recursively process children
        self._process_children(node, node_id)
        
        return node_id
    
    def _process_children(self, node, parent_id):
        """
        Process all children of a node recursively
        
        This function handles all types of child nodes found in the hdlConvertor AST
        """
        # Process standard 'objs' list
        if hasattr(node, 'objs') and node.objs:
            objs_id = self._add_node("Objects", "Child Objects", "lightsteelblue")
            self.graph.edge(parent_id, objs_id)
            for obj in node.objs:
                self._process_node(obj, objs_id)
        
        # Process params (generics)
        if hasattr(node, 'params') and node.params:
            params_id = self._add_node("Params", "Generic Parameters", "gold")
            self.graph.edge(parent_id, params_id)
            for param in node.params:
                self._process_node(param, params_id)
        
        # Process ports
        if hasattr(node, 'ports') and node.ports:
            ports_id = self._add_node("Ports", "Port Declarations", "lightgreen")
            self.graph.edge(parent_id, ports_id)
            for port in node.ports:
                self._process_node(port, ports_id)
        
        # Process module name for definitions
        if hasattr(node, 'module_name') and node.module_name:
            if isinstance(node.module_name, str):
                module_id = self._add_node("ModuleName", f"Module: {node.module_name}", "lightskyblue")
                self.graph.edge(parent_id, module_id)
            else:
                self._process_node(node.module_name, parent_id)
        
        # Process entity declarations
        if hasattr(node, 'dec') and node.dec:
            self._process_node(node.dec, parent_id)
        
        # Process entity definitions
        if hasattr(node, 'def_') and node.def_:
            self._process_node(node.def_, parent_id)
        
        # Process expressions like conditions, values, etc.
        for attr in ['cond', 'value', 'src', 'dst', 'left', 'right', 'op', 'type']:
            if hasattr(node, attr) and getattr(node, attr) is not None:
                value = getattr(node, attr)
                # Only process objects, not primitives
                if hasattr(value, '__dict__') or isinstance(value, (list, dict)):
                    attr_id = self._add_node(attr.capitalize(), f"{attr}", "lightcoral")
                    self.graph.edge(parent_id, attr_id)
                    self._process_node(value, attr_id)
        
        # Process parameter mappings for component instances
        if hasattr(node, 'param_map') and node.param_map:
            param_map_id = self._add_node("ParamMap", "Generic Mapping", "gold")
            self.graph.edge(parent_id, param_map_id)
            for mapping in node.param_map:
                self._process_node(mapping, param_map_id)
        
        # Process port mappings for component instances
        if hasattr(node, 'port_map') and node.port_map:
            port_map_id = self._add_node("PortMap", "Port Mapping", "lightgreen")
            self.graph.edge(parent_id, port_map_id)
            for mapping in node.port_map:
                self._process_node(mapping, port_map_id)
        
        # Process statement bodies (like process statements)
        if hasattr(node, 'body') and node.body:
            if hasattr(node.body, '__dict__') or isinstance(node.body, (list, dict)):
                body_id = self._add_node("Body", "Statement Body", "lightgrey")
                self.graph.edge(parent_id, body_id)
                if isinstance(node.body, list):
                    for item in node.body:
                        self._process_node(item, body_id)
                else:
                    self._process_node(node.body, body_id)
        
        # Process if-else structures
        if hasattr(node, 'if_true') and node.if_true:
            if_id = self._add_node("IfTrue", "If Branch", "lightgrey")
            self.graph.edge(parent_id, if_id)
            self._process_node(node.if_true, if_id)
            
        if hasattr(node, 'elifs') and node.elifs:
            for i, elif_branch in enumerate(node.elifs):
                elif_id = self._add_node(f"Elif_{i}", f"Elif Branch {i}", "lightgrey")
                self.graph.edge(parent_id, elif_id)
                self._process_node(elif_branch, elif_id)
        
        # Process lists of items
        if isinstance(node, list):
            for i, item in enumerate(node):
                self._process_node(item, parent_id)
        
        # Process operator operands
        if hasattr(node, 'ops') and node.ops:
            ops_id = self._add_node("Operands", "Operator Operands", "lightsalmon")
            self.graph.edge(parent_id, ops_id)
            for op in node.ops:
                self._process_node(op, ops_id)
    
    def _create_node_label(self, node):
        """Create a readable label for a node with its key attributes"""
        parts = []
        
        # Add class name
        if hasattr(node, '__class__') and hasattr(node.__class__, '__name__'):
            parts.append(node.__class__.__name__)
        
        # Add name if available
        if hasattr(node, 'name'):
            if hasattr(node.name, 'val'):
                parts.append(f"name: {node.name.val}")
            else:
                parts.append(f"name: {node.name}")
        
        # Add value if available
        if hasattr(node, 'val') and node.val is not None:
            parts.append(f"val: {node.val}")
        
        # Add operator for binary/unary operations
        if hasattr(node, 'fn') and node.fn is not None:
            parts.append(f"fn: {node.fn}")
        
        # Add direction for ports
        if hasattr(node, 'direction') and node.direction is not None:
            parts.append(f"direction: {node.direction}")
        
        # Add type information
        if hasattr(node, 'type') and node.type is not None:
            if isinstance(node.type, str):
                parts.append(f"type: {node.type}")
            elif hasattr(node.type, 'name'):
                parts.append(f"type: {node.type.name}")
            else:
                parts.append(f"type: {type(node.type).__name__}")
        
        # If it's just a string or simple type, use its value
        if isinstance(node, (str, int, float, bool)):
            return str(node)
            
        # Handle lists by showing their length
        if isinstance(node, list):
            return f"List[{len(node)} items]"
        
        # If no special attributes found, use repr
        if len(parts) == 0:
            return str(type(node).__name__)
        
        return "\n".join(parts)
    
    def _get_node_color(self, node_type):
        """Determine color based on node type"""
        if 'Library' in node_type or 'Import' in node_type:
            return "lightblue"
        elif 'Module' in node_type or 'Entity' in node_type:
            return "skyblue"
        elif 'Port' in node_type:
            return "lightgreen"
        elif 'Param' in node_type or 'Generic' in node_type:
            return "gold"
        elif 'Expr' in node_type or 'Op' in node_type:
            return "lightsalmon"
        elif 'Value' in node_type:
            return "lightcyan"
        elif 'Stm' in node_type or 'Process' in node_type:
            return "lightpink"
        elif 'Comp' in node_type:
            return "plum"
        else:
            return "lightgrey"
    
    def _add_node(self, node_type, label, color):
        """Add a node to the graph with appropriate styling"""
        node_id = f"node_{self.node_counter}"
        self.node_counter += 1
        self.graph.node(node_id, label, shape='box', style='filled', fillcolor=color)
        return node_id


def visualize_vhdl_file(file_path, output_file=None):
    """
    Visualize the hierarchical structure of a VHDL file
    with full recursive traversal of all AST elements
    """
    if output_file is None:
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = f"{base_name}_ast"
        
    visualizer = VhdlAstVisualizer()
    ast = visualizer.parse_file(file_path)
    return visualizer.visualize(ast, output_file)


In [90]:
# Example usage
if __name__ == "__main__":
    # Visualize from a file
    visualize_vhdl_file("fft_r2_par.vhd")

Graph rendered successfully to fft_r2_par_ast.svg


fft_r2_par.vhd:171:4: VhdlProcessParser.visitAttribute_declaration Conversion to Python object not implemented
    ...attributekeep_hierarchy:string;...
fft_r2_par.vhd:172:4: VhdlProcessParser.visitAttribute_specification Conversion to Python object not implemented
    ...attributekeep_hierarchyofstr:architectureis"yes";...


In [105]:
class ComprehensiveVhdlAstVisualizer:
    """
    Visualize complete VHDL AST hierarchy including all leaf nodes
    """
    def __init__(self):
        self.graph = graphviz.Digraph(
            'Complete VHDL AST', 
            comment='Full VHDL AST with Leaf Nodes',
            format='svg'
        )
        
        # Configuration for better readability
        self.graph.attr(
            rankdir='LR',          # Left to right layout for better hierarchy view
            nodesep='0.3',         # Reduced node separation
            ranksep='0.5',         # Good rank separation
            fontname='Arial',      # Clean font
            dpi='30'              # Higher resolution
        )
        
        # Node styling defaults
        self.graph.attr('node', 
                      shape='box', 
                      style='filled,rounded', 
                      margin='0.2',
                      fontsize='10',
                      fontname='Arial')
        
        # Edge styling defaults
        self.graph.attr('edge',
                      fontsize='8',
                      fontname='Arial',
                      arrowsize='0.6')
        
        # Clean color palette
        self.colors = {
            'module':    '#42a5f5',  # Blue
            'entity':    '#26a69a',  # Teal
            'port':      '#66bb6a',  # Green
            'generic':   '#ffca28',  # Amber
            'expression':'#ef5350',  # Red
            'primitive': '#78909c',  # Blue Grey
            'leaf':      '#e0e0e0',  # Light Grey
            'list':      '#9575cd',  # Purple
            'default':   '#bdbdbd'   # Grey
        }
        
        self.node_counter = 0
        self.seen_nodes = {}  # Avoid cycles
        self.max_label_len = 50  # Limit label length
        
    def parse_file(self, file_path):
        """Parse a VHDL file and return the AST"""
        converter = HdlConvertor()
        ast = converter.parse([file_path], Language.VHDL_2008, [])
        return ast
    
    def visualize(self, ast, output_file="vhdl_complete_ast"):
        """Create visualization of the complete AST including leaf nodes"""
        # Add root node
        root_id = self._add_node("HdlContext", "VHDL Design Context", self.colors['default'])
        
        # Process all top-level objects
        if hasattr(ast, 'objs') and ast.objs:
            for obj in ast.objs:
                self._process_node(obj, root_id)
                
        # Render the graph
        try:
            self.graph.render(output_file, view=False)
            print(f"Complete AST rendered to {output_file}.svg")
            return f"{output_file}.svg"
        except Exception as e:
            print(f"Error rendering graph: {e}")
            return None
    
    def _process_node(self, node, parent_id):
        """
        Recursively process any node in the AST
        
        Args:
            node: Current AST node to process
            parent_id: ID of parent node in visualization
        """
        # Skip if node already processed
        node_hash = id(node)
        if node_hash in self.seen_nodes:
            self.graph.edge(parent_id, self.seen_nodes[node_hash], style="dashed")
            return self.seen_nodes[node_hash]
        
        # Handle primitive types (leaf nodes)
        if self._is_primitive(node):
            node_id = self._add_leaf_node(node, self.colors['leaf'])
            self.seen_nodes[node_hash] = node_id
            self.graph.edge(parent_id, node_id)
            return node_id
        
        # Get node type and create label
        node_type = self._get_node_type(node)
        label = self._create_node_label(node)
        color = self._get_node_color(node_type)
        
        # Add node to graph
        node_id = self._add_node(node_type, label, color)
        self.seen_nodes[node_hash] = node_id
        
        # Connect to parent
        self.graph.edge(parent_id, node_id)
        
        # Process all attributes (going to leaf nodes)
        if isinstance(node, dict):
            self._process_dict_attributes(node, node_id)
        elif isinstance(node, list):
            self._process_list_items(node, node_id)
        else:
            self._process_object_attributes(node, node_id)
        
        return node_id
    
    def _process_object_attributes(self, node, parent_id):
        """Process all attributes of an object"""
        for attr_name in dir(node):
            # Skip special attributes and methods
            if attr_name.startswith('__') or callable(getattr(node, attr_name)):
                continue
            
            try:
                attr_value = getattr(node, attr_name)
                
                # Skip None values
                if attr_value is None:
                    continue
                
                if self._is_primitive(attr_value):
                    # Add leaf node for primitive values
                    leaf_id = self._add_leaf_node(f"{attr_name}: {attr_value}", self.colors['primitive'])
                    self.graph.edge(parent_id, leaf_id, label=attr_name)
                else:
                    # Process non-primitive attribute
                    child_id = self._process_node(attr_value, parent_id)
                    self.graph.edge(parent_id, child_id, label=attr_name)
            except Exception:
                # Skip attributes that cannot be accessed
                pass
    
    def _process_dict_attributes(self, node_dict, parent_id):
        """Process all items in a dictionary"""
        for key, value in node_dict.items():
            if value is None:
                continue
                
            if self._is_primitive(value):
                # Add leaf node for primitive values
                leaf_id = self._add_leaf_node(f"{key}: {value}", self.colors['primitive'])
                self.graph.edge(parent_id, leaf_id, label=str(key))
            else:
                # Process non-primitive value
                child_id = self._process_node(value, parent_id)
                self.graph.edge(parent_id, child_id, label=str(key))
    
    def _process_list_items(self, node_list, parent_id):
        """Process all items in a list"""
        # For large lists, use a container
        if len(node_list) > 10:
            list_container_id = self._add_node("List", f"List with {len(node_list)} items", self.colors['list'])
            self.graph.edge(parent_id, list_container_id, label="items")
            
            # Process a subset of items
            for i, item in enumerate(node_list[:5]):
                if self._is_primitive(item):
                    leaf_id = self._add_leaf_node(f"[{i}]: {item}", self.colors['primitive'])
                    self.graph.edge(list_container_id, leaf_id, label=f"[{i}]")
                else:
                    child_id = self._process_node(item, list_container_id)
                    self.graph.edge(list_container_id, child_id, label=f"[{i}]")
            
            # Add ellipsis node
            if len(node_list) > 5:
                ellipsis_id = self._add_node("More", f"... {len(node_list)-5} more items", self.colors['leaf'])
                self.graph.edge(list_container_id, ellipsis_id, style="dotted")
        else:
            # Process all items for smaller lists
            for i, item in enumerate(node_list):
                if self._is_primitive(item):
                    leaf_id = self._add_leaf_node(f"[{i}]: {item}", self.colors['primitive'])
                    self.graph.edge(parent_id, leaf_id, label=f"[{i}]")
                else:
                    child_id = self._process_node(item, parent_id)
                    self.graph.edge(parent_id, child_id, label=f"[{i}]")
    
    def _create_node_label(self, node):
        """Create a readable label for a node"""
        # For primitive types, just return string representation
        if self._is_primitive(node):
            return str(node)
        
        parts = []
        
        # Add class name
        node_type = self._get_node_type(node)
        parts.append(node_type)
        
        # Add name if available
        if hasattr(node, 'name'):
            name_str = self._extract_name_value(node.name)
            if name_str:
                parts.append(f"name: {name_str}")
        
        # Add special attributes for different node types
        if hasattr(node, 'direction'):
            parts.append(f"direction: {node.direction}")
            
        if hasattr(node, 'val') and node.val is not None:
            parts.append(f"val: {node.val}")
            
        if hasattr(node, 'fn') and node.fn is not None:
            parts.append(f"fn: {node.fn}")
        
        # Truncate if too long
        label = "\n".join(parts)
        if len(label) > self.max_label_len:
            label = label[:self.max_label_len-3] + "..."
            
        return label
    
    def _extract_name_value(self, name_obj):
        """Extract the actual name value from a name object"""
        if hasattr(name_obj, 'val'):
            return name_obj.val
        elif hasattr(name_obj, '__class__') and hasattr(name_obj.__class__, '__name__'):
            if name_obj.__class__.__name__ == 'str':
                if hasattr(name_obj, 'val'):
                    return name_obj.val
            return str(name_obj)
        else:
            return str(name_obj)
    
    def _is_primitive(self, value):
        """Check if a value is a primitive (leaf node)"""
        return isinstance(value, (str, int, float, bool)) or value is None
    
    def _get_node_type(self, node):
        """Get the type name of a node"""
        if hasattr(node, '__class__') and hasattr(node.__class__, '__name__'):
            return node.__class__.__name__
        return type(node).__name__
    
    def _get_node_color(self, node_type):
        """Determine color based on node type"""
        if 'Library' in node_type or 'Import' in node_type:
            return self.colors['module']
        elif 'Module' in node_type or 'Entity' in node_type:
            return self.colors['entity']
        elif 'Port' in node_type or 'IdDef' in node_type:
            return self.colors['port']
        elif 'Param' in node_type or 'Generic' in node_type:
            return self.colors['generic']
        elif 'Expr' in node_type or 'Op' in node_type:
            return self.colors['expression']
        elif node_type in ('str', 'int', 'bool', 'float'):
            return self.colors['primitive']
        elif node_type == 'list':
            return self.colors['list']
        else:
            return self.colors['default']
    
    def _add_node(self, node_type, label, color):
        """Add a node to the graph"""
        node_id = f"node_{self.node_counter}"
        self.node_counter += 1
        self.graph.node(node_id, label, fillcolor=color)
        return node_id
    
    def _add_leaf_node(self, label, color):
        """Add a leaf node (primitive value) to the graph"""
        node_id = f"leaf_{self.node_counter}"
        self.node_counter += 1
        
        if isinstance(label, (str, int, float, bool)):
            label_str = str(label)
        else:
            label_str = label
            
        # Truncate long labels
        if len(label_str) > self.max_label_len:
            label_str = label_str[:self.max_label_len-3] + "..."
            
        self.graph.node(node_id, label_str, shape='ellipse', fillcolor=color)
        return node_id


def visualize_complete_vhdl_ast(file_path, output_file=None):
    """
    Visualize the complete VHDL AST including all leaf nodes
    
    Args:
        file_path: Path to VHDL file
        output_file: Output file name (without extension)
        
    Returns:
        Path to the generated visualization
    """
    if output_file is None:
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = f"{base_name}_complete_ast"
        
    visualizer = ComprehensiveVhdlAstVisualizer()
    ast = visualizer.parse_file(file_path)
    return visualizer.visualize(ast, output_file)


In [107]:
visualize_complete_vhdl_ast(filenames[0])

Complete AST rendered to common_add_sub_complete_ast.svg


'common_add_sub_complete_ast.svg'

In [111]:

class VhdlAstVisualizer:
    """
    Visualize VHDL AST with focus on generics, conditional generation, and parameter dependencies
    """
    def __init__(self):
        self.graph = graphviz.Digraph(
            'VHDL AST', 
            comment='VHDL Module Structure with Generic Dependencies',
            format='svg'
        )
        # Improve layout with better spacing and formatting
        self.graph.attr(
            rankdir='TB',          # Top to bottom for hierarchy
            nodesep='0.5',         # Node separation
            ranksep='0.8',         # Rank separation
            fontname='Arial',      # Modern font
            splines='polyline',    # Better routing with xlabels
            concentrate='false',   # Avoid edge concentration for clarity
            newrank='true'         # Better ranking
        )
        
        # Visual palette
        self.colors = {
            'entity':    '#3949ab',  # Indigo
            'generic':   '#f57c00',  # Orange
            'port':      '#43a047',  # Green
            'instance':  '#7b1fa2',  # Purple
            'condition': '#e53935',  # Red
            'highlight': '#ef5350',  # Red for dependencies
            'default':   '#546e7a'   # Blue gray
        }
        
        # State tracking
        self.node_counter = 0
        self.seen_nodes = {}         # Avoid cycles
        self.generic_params = {}     # Track generic parameters
        self.generic_dependencies = []  # Track generic dependencies
        
    def parse_file(self, file_path):
        """Parse a VHDL file and return the AST"""
        converter = HdlConvertor()
        ast = converter.parse([file_path], Language.VHDL_2008, [])
        return ast
    
    def visualize(self, ast, output_file="vhdl_ast"):
        """Create visualization with compact nodes and generic dependency highlighting"""
        # Configure node styling
        self.graph.attr('node', 
                       shape='box', 
                       style='filled,rounded', 
                       margin='0.2',
                       fontsize='10',
                       fontname='Arial')
                      
        # Configure edge styling
        self.graph.attr('edge',
                       fontsize='9',
                       fontname='Arial',
                       arrowsize='0.6')
                      
        # Add root node
        root_id = self._add_node("HdlContext", "VHDL Design Context", self.colors['default'])
        
        # Process all top-level objects
        if hasattr(ast, 'objs') and ast.objs:
            for obj in ast.objs:
                self._process_node(obj, root_id)
        
        # Add dependency edges last to highlight them
        self._highlight_generic_dependencies()
                
        # Render the graph
        try:
            self.graph.render(output_file, view=False)
            print(f"AST visualization saved to {output_file}.svg")
            return f"{output_file}.svg"
        except Exception as e:
            print(f"Error rendering graph: {e}")
            return None
    
    def _process_node(self, node, parent_id):
        """Process an AST node with compact attribute representation"""
        # Avoid cycles
        node_hash = id(node)
        if node_hash in self.seen_nodes:
            self.graph.edge(parent_id, self.seen_nodes[node_hash], style="dashed")
            return self.seen_nodes[node_hash]
        
        # Skip creating nodes for primitives - include in parent
        if self._is_primitive(node):
            return parent_id
            
        # Create label that includes all attributes inline
        node_type = self._get_node_type(node)
        label = self._create_compact_label(node)
        color = self._get_node_color(node_type)
        
        # Add node to graph
        node_id = self._add_node(node_type, label, color)
        self.seen_nodes[node_hash] = node_id
        
        # Connect to parent
        self.graph.edge(parent_id, node_id)
        
        # Special handling for generic parameters - track them
        if self._is_generic_param(node):
            name = self._get_attribute_value(node.name)
            self.generic_params[name] = {
                'node_id': node_id,
                'type': self._get_attribute_value(node.type) if hasattr(node, 'type') else 'unknown',
                'value': self._get_attribute_value(node.value) if hasattr(node, 'value') else None
            }
        
        # Special handling for conditional generation (generate if/case)
        if self._is_conditional_generate(node):
            self._process_conditional_generation(node, node_id)
        
        # Process children (components, statements, etc.)
        self._process_children(node, node_id)
        
        return node_id
    
    def _process_children(self, node, parent_id):
        """Process child nodes with compact representation"""
        # Process core collections
        for attr, label in [
            ('params', 'Generics'),
            ('ports', 'Ports'),
            ('objs', 'Objects')
        ]:
            if hasattr(node, attr) and getattr(node, attr):
                items = getattr(node, attr)
                if items:
                    for item in items:
                        self._process_node(item, parent_id)
        
        # Process hierarchy
        for attr in ['dec', 'def_']:
            if hasattr(node, attr) and getattr(node, attr):
                self._process_node(getattr(node, attr), parent_id)
        
        # Process component instantiations
        if hasattr(node, 'param_map') and node.param_map:
            param_map_id = self._add_node("GenericMap", "Generic Mapping", self.colors['generic'])
            self.graph.edge(parent_id, param_map_id, xlabel="generic map")
            
            for mapping in node.param_map:
                map_id = self._process_node(mapping, param_map_id)
                self._check_generic_dependency(mapping, map_id)
        
        if hasattr(node, 'port_map') and node.port_map:
            port_map_id = self._add_node("PortMap", "Port Mapping", self.colors['port'])
            self.graph.edge(parent_id, port_map_id, xlabel="port map")
            
            for mapping in node.port_map:
                self._process_node(mapping, port_map_id)
        
        # Process bodies and branches
        if hasattr(node, 'body'):
            body = node.body
            if isinstance(body, list):
                for item in body:
                    self._process_node(item, parent_id)
            elif body:
                self._process_node(body, parent_id)
        
        # Process if-else structures
        for attr, label in [
            ('if_true', 'then'),
            ('if_false', 'else')
        ]:
            if hasattr(node, attr) and getattr(node, attr):
                branch_id = self._process_node(getattr(node, attr), parent_id)
                self.graph.edge(parent_id, branch_id, xlabel=label)
        
        # Process elifs
        if hasattr(node, 'elifs') and node.elifs:
            for i, elif_branch in enumerate(node.elifs):
                self._process_node(elif_branch, parent_id)
        
        # Process expressions and operands
        if hasattr(node, 'ops') and node.ops:
            for op in node.ops:
                self._process_node(op, parent_id)
    
    def _is_generic_param(self, node):
        """Check if node is a generic parameter"""
        return (self._get_node_type(node) == 'HdlIdDef' and 
                hasattr(node, 'direction') and node.direction == 'IN' and
                hasattr(node, 'name'))
        # return (self._get_node_type(node) == 'HdlIdDef' and 
        #         hasattr(node, 'direction') and node.direction == 'IN' and
        #         hasattr(node, 'name'))
    
    def _is_conditional_generate(self, node):
        """Check if node is a conditional generate statement"""
        return (hasattr(node, 'in_preproc') and node.in_preproc and
                (self._get_node_type(node) == 'HdlStmIf' or 
                 self._get_node_type(node) == 'HdlStmCase'))
    
    def _process_conditional_generation(self, node, node_id):
        """Process conditional generate statement and track generic dependencies"""
        # Format the node specially
        label = self._create_conditional_label(node)
        self.graph.node(node_id, label, style='filled,rounded,bold', 
                      fillcolor=self.colors['condition'],
                      penwidth='2.0')
        
        # Check for generic dependencies in condition
        if hasattr(node, 'cond'):
            cond_str = str(node.cond)
            generics_used = []
            
            for generic_name in self.generic_params.keys():
                if generic_name in cond_str:
                    generics_used.append(generic_name)
                    
            if generics_used:
                self.generic_dependencies.append({
                    'from': generics_used,
                    'to': node_id,
                    'type': 'conditional_generation'
                })
    
    def _check_generic_dependency(self, mapping, map_id):
        """Check if a mapping references any generic parameters"""
        mapping_str = str(mapping)
        generics_used = []
        
        for generic_name in self.generic_params.keys():
            if generic_name in mapping_str:
                generics_used.append(generic_name)
                
        if generics_used:
            self.generic_dependencies.append({
                'from': generics_used,
                'to': map_id,
                'type': 'parameter_mapping'
            })
    
    def _highlight_generic_dependencies(self):
        """Add special highlighting for generic-dependent nodes"""
        for dep in self.generic_dependencies:
            for generic_name in dep['from']:
                if generic_name in self.generic_params:
                    generic_id = self.generic_params[generic_name]['node_id']
                    
                    # Create appropriate label based on dependency type
                    label = "controls" if dep['type'] == 'conditional_generation' else "used by"
                    
                    # Add highlighted edge
                    self.graph.edge(
                        generic_id, 
                        dep['to'], 
                        xlabel=label,
                        color=self.colors['highlight'],
                        style="dashed",
                        penwidth="1.5"
                    )
    
    def _create_compact_label(self, node):
        """Create a single compact label with all attributes included"""
        parts = []
        node_type = self._get_node_type(node)
        parts.append(node_type)
        
        # Collect common attributes
        attrs = {
            'name': self._get_attribute_value(node.name) if hasattr(node, 'name') else None,
            'direction': node.direction if hasattr(node, 'direction') else None,
            'type': self._get_attribute_value(node.type) if hasattr(node, 'type') else None,
            'value': self._get_attribute_value(node.value) if hasattr(node, 'value') else None,
            'fn': node.fn if hasattr(node, 'fn') else None,
            'module': self._get_attribute_value(node.module_name) if hasattr(node, 'module_name') else None
        }
        
        # Add non-None attributes to label
        for attr, value in attrs.items():
            if value is not None:
                parts.append(f"{attr}: {value}")
        
        # Add generate statement indicator
        if hasattr(node, 'in_preproc') and node.in_preproc:
            parts.append("⚡ Generate Statement")
        
        # Truncate if too long
        if len(parts) > 6:
            parts = parts[:6] + ["..."]
            
        return "\n".join(parts)
    
    def _create_conditional_label(self, node):
        """Create specialized label for conditional generate statements"""
        parts = []
        node_type = self._get_node_type(node)
        
        if node_type == 'HdlStmIf':
            parts.append("GENERATE IF")
            if hasattr(node, 'cond'):
                condition = str(node.cond)
                if len(condition) > 60:
                    condition = condition[:57] + "..."
                parts.append(f"Condition: {condition}")
        else:
            parts.append("GENERATE CASE")
            if hasattr(node, 'switch_on'):
                parts.append(f"Switch: {node.switch_on}")
        
        # Mark generics used in condition
        if hasattr(node, 'cond'):
            cond_str = str(node.cond)
            generics_used = []
            
            for generic_name in self.generic_params.keys():
                if generic_name in cond_str:
                    generics_used.append(generic_name)
                    
            if generics_used:
                parts.append(f"Used generics: {', '.join(generics_used)}")
        
        return "\n".join(parts)
    
    def _get_attribute_value(self, attr):
        """Extract a readable value from an attribute"""
        if attr is None:
            return "None"
            
        # Handle string value objects
        if hasattr(attr, 'val'):
            return attr.val
            
        # Handle expressions with function name and operands
        if hasattr(attr, 'fn') and hasattr(attr, 'ops'):
            return f"{attr.fn}(...)"
            
        # Default to string representation
        return str(attr)
    
    def _is_primitive(self, value):
        """Check if a value is a primitive (leaf node)"""
        return isinstance(value, (str, int, float, bool)) or value is None
    
    def _get_node_type(self, node):
        """Get the type name of a node"""
        if hasattr(node, '__class__') and hasattr(node.__class__, '__name__'):
            return node.__class__.__name__
        return type(node).__name__
    
    def _get_node_color(self, node_type):
        """Determine node color based on type"""
        if 'Module' in node_type or 'Entity' in node_type:
            return self.colors['entity']
        elif 'Port' in node_type or 'IdDef' in node_type and hasattr(node_type, 'direction') and node_type.direction in ['IN', 'OUT', 'INOUT']:
            return self.colors['port']
        elif 'Param' in node_type or 'Generic' in node_type:
            return self.colors['generic']
        elif 'Comp' in node_type or 'Inst' in node_type:
            return self.colors['instance']
        else:
            return self.colors['default']
    
    def _add_node(self, node_type, label, color):
        """Add a node to the graph with appropriate styling"""
        node_id = f"node_{self.node_counter}"
        self.node_counter += 1
        
        self.graph.node(
            node_id, 
            label, 
            style='filled,rounded', 
            fillcolor=color,
            color="#333333"  # Darker border
        )
        return node_id


def visualize_vhdl_ast(file_path, output_file=None):
    """
    Visualize a VHDL file with focus on generics, ports and their dependencies
    
    Args:
        file_path: Path to VHDL file
        output_file: Output file path (without extension)
    
    Returns:
        Path to the generated visualization
    """
    if output_file is None:
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = f"{base_name}_ast"
        
    visualizer = VhdlAstVisualizer()
    ast = visualizer.parse_file(file_path)
    return visualizer.visualize(ast, output_file)


In [112]:
visualize_vhdl_ast(filenames[1])

AST visualization saved to fft_r2_par_ast.svg


fft_r2_par.vhd:171:4: VhdlProcessParser.visitAttribute_declaration Conversion to Python object not implemented
    ...attributekeep_hierarchy:string;...
fft_r2_par.vhd:172:4: VhdlProcessParser.visitAttribute_specification Conversion to Python object not implemented
    ...attributekeep_hierarchyofstr:architectureis"yes";...


'fft_r2_par_ast.svg'

In [ ]:
for obj in ast.objs:
    print(dir(obj))
    if hasattr(obj, 'dec') and obj.dec is not None:
        print(obj)

In [ ]:
attrs = set()
types = set()
for obj in ast.objs:
    types.add(type(obj))
    if isinstance(obj, hdlConvertorAst.hdlAst._structural.HdlModuleDec):
        # print(obj)
        ...
    elif isinstance(obj, hdlConvertorAst.hdlAst._structural.HdlModuleDef):
        print(obj)
        ...
types

In [ ]:
from hdlConvertorAst.to.verilog import ToVerilog2005
TEST_DIR = os.path.join("..", "tests", "verilog")

filenames = [os.path.join(TEST_DIR, "fifo_rx.v"), ]
include_dirs = []

ast = HdlConvertor().parse(
    filenames=filenames[0],
    language=Language.VERILOG_2005,
    incdirs=include_dirs,
    hierarchyOnly=False
)
PortNamesToLowerCase().visit_HdlContext(ast)
serializer = ToVerilog2005(sys.stdout)
code = serializer.visit_HdlContext(ast)
code;

In [ ]:
for obj in ast.objs:
    if hasattr(obj, "doc") and obj.doc:
        print(f"{type(obj).__name__} has doc: {obj.doc}")


In [ ]:
# Example usage
vhdl_file = filenames[0]

# Extract generics
generics = print_vhdl_generics(vhdl_file, "vhdl_generics.json")

# Example of modifying generics
if generics:
    entity_name = next(iter(generics.keys()))
    modified_generics = {
        entity_name: {
            "WIDTH": 32,  # Change WIDTH to 32
            "RESET_ACTIVE": "'0'"  # Change RESET_ACTIVE to '0'
        }
    }
    modify_vhdl_generics(vhdl_file, modified_generics, "modified_" + vhdl_file)

In [ ]:
modifer.ast.objs;

In [ ]:
tv = ToVhdl2008(sys.stdout)
tv.visit_HdlContext(modifer.ast)
